In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import random
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [ ]:
df = pd.read_csv('../small_dataset_csvs/events_with_number_of_frames.csv')

In [ ]:
df1 = df[['event_path','exposure','burial','field_joint','anode','free_span']]

In [ ]:
fig1, ax1 = plt.subplots()
df1.iloc[:,1:].sum(axis=0).plot.pie(autopct='%1.1f%%',shadow=True, startangle=90,ax=ax1)
ax1.axis("equal")
plt.show()

In [ ]:
df1.iloc[:,1:].sum(axis=0)

In [ ]:
df

In [ ]:
len(df)

In [ ]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:
seq_length = 20

In [ ]:
df = df[df.number_of_frames>=seq_length] # & df.number_of_frames <= 500]
len(df)

In [ ]:
df = df[df.number_of_frames<=500]

In [ ]:
len(df)

In [ ]:
fig1, ax1 = plt.subplots()
df1.iloc[:,1:].sum(axis=0).plot.pie(autopct='%1.1f%%',shadow=True, startangle=90,ax=ax1)
ax1.axis("equal")
plt.show()

In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
df['fold'] = 0

In [ ]:
g = df.groupby('label')

In [ ]:
i = g['number_of_frames'].quantile([0.05, 0.25, 0.5, 0.9]).unstack()
j = g['number_of_frames'].agg(['min', 'max'])

In [ ]:
pd.concat([i, j], 1)

In [ ]:
values = [111, 80.0, 34.0, 42.0, 79]

In [ ]:
i.T.plot(subplots=True)
plt.show()

In [ ]:
df['stratify_group'] = 'yo'

In [ ]:
labels = df.label.unique()

In [ ]:
for l, v in zip(labels, values):
    df.loc[df.label==l, 'stratify_group'] = np.char.add(
        df.loc[df.label==l, 'label'].values.astype(str),
        df.loc[df.label==l, 'number_of_frames'].apply(lambda x: f'_{int(x // v)}').values.astype(str)
    )

In [ ]:
df

In [ ]:
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df['event_path'], y=df['stratify_group'])):
    df.loc[df.iloc[val_index].index, 'fold'] = fold_number

In [ ]:
df

In [ ]:
df.to_csv('../small_dataset_csvs/events_with_number_of_frames_stratified.csv', index=False)

In [ ]:
df.fold.value_counts()

In [ ]:
df.stratify_group.value_counts()

In [ ]:
df = df[df['number_of_frames']>20]